In [7]:
# from geopy.distance import vincenty as latlongdist
import geopy.distance
from helper import *
import ast
import random

#Read date
bounds_raw = pd.read_csv('adjacencyGeoms.csv',index_col=None)
bounds_raw = bounds_raw#.iloc[:50,:]
idx = bounds_raw['geom'].apply(len)>2
bounds_raw = bounds_raw.loc[idx]

In [8]:
# Process data
def vtd_idx(row):
    low_idx = inv_vtds[row['low']]
    high_idx = inv_vtds[row['high']]
    if low_idx > high_idx:
        low_idx, high_idx = high_idx, low_idx

    pts = np.array(ast.literal_eval(row['geom'])[0])    
    pts = pts[:,:2]    
    pairs = zip(pts,pts[1:])
    length = 0
    for (xi,xf) in pairs:
        # geopy.distance.vincenty is a more accurate but slow way to convert lat-long difference into distance
#         dl = geopy.distance.vincenty(xi,xf).meters

        # this is standard Euclidean distance - fast but not as accurate
        dx = xf-xi
        dl = np.sqrt(dx.dot(dx))
        length += dl
    return [low_idx, high_idx, (low_idx,high_idx), length, vtd_geoids[low_idx], vtd_geoids[high_idx], pts]

vtd_geoids = np.unique(bounds_raw[['low','high']]).tolist()
num_vtds = len(vtd_geoids)
inv_vtds = {vtd: i for i, vtd in enumerate(vtd_geoids)}

new = [vtd_idx(row[1]) for row in bounds_raw.iterrows()]

bounds = pd.DataFrame.from_records(new,columns=['low_idx','high_idx','pair','length','low_geo','high_geo','pts'])
bounds.sort_values('pair',inplace=True)
bounds.reset_index(inplace=True)
# bounds.set_index('pair',inplace=True)

pop=np.random.randint(100000,size=num_vtds)

G = nx.Graph()
for (i,vtd) in enumerate(vtd_geoids):
    G.add_node(i, geoid=vtd, pop=pop[i])
for (b, attr) in bounds.iterrows():
    if attr['pair'] in G.edges:
        G.edges[attr['pair']]['length'] += attr['length']
    else:
        G.add_edge(*attr['pair'], length=attr['length'])

In [9]:
# initial district assignment
num_dists = 3

if num_dists > num_vtds:
    raise Exception('More districts than vtds', num_dist, num_vtds)

vtds_left = set(range(num_vtds))
dist_vtds = [[] for _ in range(num_dists)]  #list of lists of vtds in each district
vtd_dist = np.empty(num_vtds).astype(int)  #list of dist assigned to each vtd
dist_pop = np.zeros(num_dists).astype(int)

def update(dist, vtds):
    global dist_vtds, vtds_left, dist_pop, G
    dist_vtds[dist].extend(vtds)  # put new vtds in the dist list
    vtds_left.difference_update(vtds)  # remove from unassigned vtds
    for v in vtds:
        vtd_dist[v] = dist  
        G.nodes[v]['dist'] = dist  # record in graph object        
        dist_pop[dist] += pop[v]  # increase dist population

# First vtd for each district
for dist in range(num_dists):
    v = random.sample(vtds_left, 1)
    update(dist, v)
    
# grow districts
while bool(vtds_left):
    done = False
    for dist in dist_pop.argsort():  # Add to district with smallest population if possible
        vtds_in = dist_vtds[dist].copy()
        random.shuffle(vtds_in)  # Grow from a random vtd in the dist
        for v in vtds_in:
            neighbors_left = set(G.neighbors(v)).intersection(vtds_left)  # find avaiable neighboring vtds
            if bool(neighbors_left):  # if some, great.  Add all of them to the district and turn the while loop.  Else, try again
                update(dist, neighbors_left)
                done = True
                break
        if done == True:
            break
    if done == False:
        raise Exception("No place to put %s" % vtds_left)
        
bounds['low_dist'] = vtd_dist[bounds['low_idx']]
bounds['high_dist'] = vtd_dist[bounds['high_idx']]

In [10]:
# part of a step of the markov chain
# to add - does this make the map better or worse
# I have not coded any goodness calculations yet

cross = bounds['low_dist'] != bounds['high_dist']
e = random.choice(cross.index)
if random.getrandbits(1):
    vtd_move = bounds.loc[e,'low_idx']
    dist_new = bounds.loc[e,'high_dist']    
else:
    vtd_move = bounds.loc[e,'high_idx']
    dist_new = bounds.loc[e,'low_dist']

dist_left = vtd_dist[vtd_move]
dist_vtds[dist_left].remove(vtd_move)
dist_vtds[dist_new].append(vtd_move)
vtd_dist[vtd_move] = dist_new

pop[dist_left] -= pop[vtd_move]
pop[dist_new] += pop[vtd_move]
bounds['low_dist'] = vtd_dist[bounds['low_idx']]
bounds['high_dist'] = vtd_dist[bounds['high_idx']]


# end of program so far

In [11]:
print(vtds_left)
print(dist_pop)
for (vtds,pop) in zip(dist_vtds,dist_pop):
    print(pop)
    print(vtds)


set()
[35764364 54331085 44679200]
35764364
[106, 111, 114, 1874, 1878, 1879, 1880, 120, 1876, 1881, 1875, 1877, 365, 358, 554, 548, 549, 551, 379, 1370, 371, 550, 361, 370, 372, 382, 116, 113, 1283, 108, 1278, 1280, 1284, 107, 117, 126, 354, 364, 366, 553, 555, 1367, 355, 1821, 374, 536, 535, 1368, 537, 539, 356, 351, 359, 2561, 2565, 2566, 2287, 2288, 2290, 1473, 115, 1476, 2564, 561, 538, 1820, 1814, 552, 556, 543, 544, 545, 560, 557, 558, 559, 1827, 122, 124, 125, 109, 110, 119, 546, 547, 540, 542, 352, 381, 373, 380, 360, 353, 118, 1952, 1947, 112, 1942, 123, 1948, 1951, 1818, 1822, 377, 349, 1376, 1392, 541, 368, 369, 626, 2285, 2286, 1279, 1812, 1815, 1281, 635, 623, 632, 1825, 1810, 1828, 1366, 367, 362, 1808, 1819, 2562, 2563, 375, 1937, 1938, 363, 1809, 1830, 2289, 1918, 1916, 1894, 1955, 1824, 376, 378, 357, 1282, 350, 1953, 1956, 1936, 1972, 1973, 983, 132, 628, 621, 471, 1811, 696, 698, 699, 1823, 347, 619, 1892, 466, 1917, 618, 346, 613, 615, 1369, 694, 1397, 1365, 1390, 

In [12]:
for (node, attr) in G.nodes.items():
    print(node, attr)

for (edge, attr) in G.edges.items():
    print(edge, attr)

0 {'geoid': '3700101', 'pop': 59199, 'dist': 2}
1 {'geoid': '3700102', 'pop': 87539, 'dist': 2}
2 {'geoid': '37001035', 'pop': 95340, 'dist': 2}
3 {'geoid': '3700103C', 'pop': 28560, 'dist': 2}
4 {'geoid': '3700103N', 'pop': 65729, 'dist': 2}
5 {'geoid': '3700103S', 'pop': 32599, 'dist': 2}
6 {'geoid': '3700103W', 'pop': 82581, 'dist': 2}
7 {'geoid': '3700104', 'pop': 17628, 'dist': 2}
8 {'geoid': '3700105', 'pop': 26770, 'dist': 2}
9 {'geoid': '37001063', 'pop': 19260, 'dist': 2}
10 {'geoid': '37001064', 'pop': 5264, 'dist': 2}
11 {'geoid': '3700106E', 'pop': 70355, 'dist': 2}
12 {'geoid': '3700106N', 'pop': 24638, 'dist': 2}
13 {'geoid': '3700106S', 'pop': 45274, 'dist': 2}
14 {'geoid': '3700106W', 'pop': 73271, 'dist': 2}
15 {'geoid': '3700107', 'pop': 2888, 'dist': 2}
16 {'geoid': '3700108N', 'pop': 85747, 'dist': 2}
17 {'geoid': '3700108S', 'pop': 7227, 'dist': 2}
18 {'geoid': '3700109N', 'pop': 92126, 'dist': 2}
19 {'geoid': '3700109S', 'pop': 53802, 'dist': 2}
20 {'geoid': '3700

1451 {'geoid': '3711309', 'pop': 27869, 'dist': 1}
1452 {'geoid': '3711310', 'pop': 14440, 'dist': 1}
1453 {'geoid': '3711311', 'pop': 47936, 'dist': 1}
1454 {'geoid': '3711312', 'pop': 55803, 'dist': 1}
1455 {'geoid': '3711313', 'pop': 75654, 'dist': 1}
1456 {'geoid': '3711314', 'pop': 82148, 'dist': 1}
1457 {'geoid': '3711315', 'pop': 97395, 'dist': 1}
1458 {'geoid': '37115BEECH', 'pop': 24133, 'dist': 1}
1459 {'geoid': '37115EBBS-C', 'pop': 628, 'dist': 1}
1460 {'geoid': '37115GRAPEV', 'pop': 31838, 'dist': 1}
1461 {'geoid': '37115HOT-SP', 'pop': 81680, 'dist': 1}
1462 {'geoid': '37115LAUREL', 'pop': 3048, 'dist': 1}
1463 {'geoid': '37115MARS-H', 'pop': 67103, 'dist': 1}
1464 {'geoid': '37115NORTH', 'pop': 15371, 'dist': 1}
1465 {'geoid': '37115REVERE', 'pop': 85067, 'dist': 1}
1466 {'geoid': '37115SANDY', 'pop': 90302, 'dist': 1}
1467 {'geoid': '37115SOUTH', 'pop': 41776, 'dist': 1}
1468 {'geoid': '37115SPRING', 'pop': 58984, 'dist': 1}
1469 {'geoid': '37115WALNUT', 'pop': 7639, 'd

(1, 10) {'length': 0.021080056923250195}
(1, 15) {'length': 0.0628889448675848}
(1, 30) {'length': 0.05668327092932378}
(1, 1077) {'length': 0.046457855360964646}
(1, 1132) {'length': 0.03290251949774455}
(2, 3) {'length': 0.021614587406625266}
(2, 4) {'length': 0.03196069685493888}
(2, 6) {'length': 0.09292306063371994}
(2, 7) {'length': 0.11941636572744285}
(2, 25) {'length': 0.0007102478440685397}
(2, 28) {'length': 0.021815703518733146}
(2, 1145) {'length': 0.01584911719394492}
(3, 4) {'length': 0.07314439145018314}
(3, 5) {'length': 0.04933576477907703}
(3, 6) {'length': 0.0009703383057547022}
(3, 25) {'length': 0.031681932593283134}
(3, 27) {'length': 0.007054519246861889}
(4, 6) {'length': 0.06474404796491284}
(5, 6) {'length': 0.029676361750815897}
(5, 26) {'length': 0.0220599771996323}
(5, 27) {'length': 0.014625532850668846}
(5, 30) {'length': 0.05959319051952673}
(5, 1132) {'length': 0.0354437793651069}
(6, 1076) {'length': 0.045074483267627856}
(6, 1145) {'length': 0.009737

(370, 371) {'length': 0.07193648967717645}
(370, 380) {'length': 0.08108485857685596}
(370, 550) {'length': 0.05192466786153584}
(370, 551) {'length': 0.059092377557401}
(371, 379) {'length': 0.04395748276407501}
(371, 380) {'length': 0.0038533102422324806}
(371, 550) {'length': 0.05773963790171642}
(371, 554) {'length': 0.034465847497311096}
(372, 382) {'length': 0.18485125386159684}
(372, 551) {'length': 0.00316822232041582}
(373, 377) {'length': 0.07688120041319471}
(373, 381) {'length': 0.03405444981707969}
(374, 379) {'length': 0.27699374640045193}
(374, 1821) {'length': 0.011429798749192362}
(374, 1827) {'length': 0.031131517702231645}
(376, 378) {'length': 0.21531924438926472}
(377, 381) {'length': 0.0846340573991149}
(379, 554) {'length': 0.13294148103663986}
(379, 1370) {'length': 0.1264673063553988}
(379, 1821) {'length': 0.10641833289319341}
(382, 551) {'length': 0.05569157120076401}
(383, 384) {'length': 0.26029441660457026}
(383, 389) {'length': 0.023802164163184012}
(383,

(776, 784) {'length': 0.007303275251555069}
(776, 785) {'length': 0.018918150936150863}
(776, 1726) {'length': 0.08696987311840573}
(776, 1740) {'length': 0.027734979045865634}
(777, 778) {'length': 0.09575171333090338}
(777, 779) {'length': 0.1509220658390372}
(777, 780) {'length': 0.13725986541499766}
(777, 1943) {'length': 0.0507871205085933}
(777, 1946) {'length': 0.023296054251056134}
(778, 779) {'length': 0.10800268157036239}
(778, 1946) {'length': 0.09320439107070616}
(778, 2647) {'length': 0.021848361184471476}
(778, 2649) {'length': 0.07899023653950303}
(779, 780) {'length': 0.010262978033473293}
(779, 786) {'length': 0.24965532869468923}
(779, 2647) {'length': 0.0349642287343548}
(780, 783) {'length': 0.08415995296263472}
(780, 786) {'length': 0.09876777075891366}
(781, 782) {'length': 0.019951736533470394}
(781, 783) {'length': 0.04692381126037211}
(781, 784) {'length': 0.009806680326025665}
(781, 785) {'length': 0.018631938953522116}
(782, 783) {'length': 0.0536560165624426

(1190, 1206) {'length': 0.07172258587130521}
(1190, 1207) {'length': 0.09677139588408193}
(1190, 1214) {'length': 0.08105262579708122}
(1190, 2266) {'length': 0.2614042325048579}
(1191, 1192) {'length': 0.0481710752462908}
(1191, 1193) {'length': 0.00847730156699921}
(1191, 1196) {'length': 0.03363661374119371}
(1191, 1198) {'length': 0.0074620593713148105}
(1191, 1199) {'length': 0.08794857335382646}
(1191, 1209) {'length': 0.02988248109797428}
(1191, 1210) {'length': 0.013236170305038424}
(1192, 1193) {'length': 0.09655624939591653}
(1192, 1198) {'length': 0.04096668360881896}
(1192, 1200) {'length': 0.04765518241360168}
(1193, 1194) {'length': 0.10499872928555776}
(1193, 1196) {'length': 0.005809600391639078}
(1193, 1200) {'length': 0.05334160731025765}
(1194, 1196) {'length': 0.07153875240994112}
(1195, 1196) {'length': 0.10099234895170378}
(1195, 1201) {'length': 0.0116048347689222}
(1195, 1210) {'length': 0.05067980479273467}
(1196, 1210) {'length': 0.019855952635989764}
(1197, 1

(1622, 1660) {'length': 0.012449653914751151}
(1622, 1665) {'length': 0.02479872272143249}
(1623, 1631) {'length': 0.00577884770788448}
(1623, 1646) {'length': 0.08956553794264417}
(1623, 1671) {'length': 0.025889977257992462}
(1624, 1625) {'length': 0.05142582391721229}
(1624, 1636) {'length': 0.004012728836567823}
(1624, 1643) {'length': 0.05310117930269841}
(1624, 1674) {'length': 0.06918102655915141}
(1624, 1676) {'length': 0.03936371980459426}
(1625, 1641) {'length': 0.00831274610811327}
(1626, 1660) {'length': 0.02488538699051101}
(1626, 1666) {'length': 0.018755119384681426}
(1627, 1633) {'length': 0.010535012813965421}
(1627, 1646) {'length': 0.023524559412226823}
(1627, 1673) {'length': 0.04550486099937129}
(1627, 1675) {'length': 0.05403645891284365}
(1628, 1646) {'length': 0.024744944779226303}
(1628, 1648) {'length': 0.011114536972747465}
(1630, 1665) {'length': 0.009969358800543376}
(1630, 2320) {'length': 0.0424169023729307}
(1631, 1638) {'length': 0.03393529109820499}
(1

(2142, 2144) {'length': 0.03639501438940704}
(2142, 2146) {'length': 0.11180116531854394}
(2142, 2151) {'length': 0.17409048569379337}
(2142, 2154) {'length': 0.04376474866415103}
(2143, 2149) {'length': 0.12498688244595943}
(2143, 2150) {'length': 0.11351902934286576}
(2144, 2147) {'length': 0.03719191454156795}
(2144, 2151) {'length': 0.11640209900720545}
(2144, 2154) {'length': 0.027206970278085354}
(2144, 2155) {'length': 0.048294088458581706}
(2144, 2156) {'length': 0.09100943684169312}
(2145, 2146) {'length': 0.07970790640652917}
(2146, 2148) {'length': 0.050213738574584055}
(2146, 2154) {'length': 0.0033749459542873183}
(2147, 2148) {'length': 0.10442008974332892}
(2147, 2153) {'length': 0.026846155843934436}
(2147, 2154) {'length': 0.023776680060803572}
(2147, 2155) {'length': 0.17649697292179561}
(2148, 2154) {'length': 0.08672983443749342}
(2149, 2150) {'length': 0.18749683126124791}
(2149, 2152) {'length': 0.02572121648746205}
(2149, 2153) {'length': 0.033519380921135496}
(2

(2647, 2651) {'length': 0.03272636528814047}
(2647, 2653) {'length': 0.14014823229014403}
(2647, 2665) {'length': 0.05702490278377164}
(2647, 2668) {'length': 0.02824483895058371}
(2648, 2650) {'length': 0.15343466223571028}
(2648, 2652) {'length': 0.09076767727715584}
(2648, 2661) {'length': 0.03052378725940274}
(2649, 2651) {'length': 0.17717484759423308}
(2651, 2659) {'length': 0.03549908941612643}
(2651, 2665) {'length': 0.0339423038246396}
(2652, 2653) {'length': 0.0919394134118315}
(2652, 2661) {'length': 0.011486859275730656}
(2652, 2663) {'length': 0.03283581355908899}
(2652, 2664) {'length': 0.03734446711408368}
(2652, 2666) {'length': 0.03701856086073216}
(2653, 2658) {'length': 0.03293825985838049}
(2653, 2664) {'length': 0.011951382041592986}
(2653, 2668) {'length': 0.059888934674364944}
(2654, 2655) {'length': 0.00982349642886}
(2654, 2656) {'length': 0.014214784354767751}
(2654, 2657) {'length': 0.016598141356110412}
(2654, 2658) {'length': 0.019456732661355492}
(2654, 26